# ETL: Universities API → SQLite (Jupyter Notebook)

Este notebook implementa um pipeline ETL que:
1. **Extrai** os dados da API pública `http://universities.hipolabs.com/search`;
2. **Transforma** e normaliza os registros;
3. **Carrega** os dados em um banco SQLite local;
4. Executa consultas SQL de exemplo demonstrando o uso do banco.

> Observação: este notebook faz requisições HTTP à API pública. Execute as células em seu ambiente local (com internet).

In [7]:
# Requisitos: instale pacotes se necessário
# Execute apenas se necessário: !pip install requests pandas tqdm
try:
    import requests, sqlite3, pandas as pd
except Exception as e:
    print('Você pode precisar instalar dependências: pip install requests pandas tqdm')
    raise

In [8]:

BASE_URL = "https://raw.githubusercontent.com/Hipo/university-domains-list/refs/heads/master/world_universities_and_domains.json"

response = requests.get(BASE_URL)
if response.status_code == 200:
    universities = response.json()
    print(f"Total de registros coletados: {len(universities)}")
else:
    raise Exception(f"Falha ao baixar o JSON: {response.status_code}")


Total de registros coletados: 10185


In [9]:
def transform_data(universities):
    return [
        (
            uni.get("name"),
            uni.get("country"),
            uni.get("state-province"),
            ",".join(uni.get("web_pages", [])),
            ",".join(uni.get("domains", []))
        )
        for uni in universities
    ]

universities_data = transform_data(universities)
print(universities_data[:5]) 

[('Engineering Institute of Technology', 'Australia', None, 'https://www.eit.edu.au/', 'student.eit.edu.au'), ('Universitas Nusa Putra', 'Indonesia', None, 'https://nusaputra.ac.id', 'nusaputra.ac.id'), ('University of Kyrenia', 'Turkey', None, 'https://kyrenia.edu.tr', 'std.kyrenia.edu.tr,kyrenia.edu.tr'), ('Regent University College of Science and Technology', 'Ghana', None, 'https://regent.edu.gh', 'regent.edu.gh'), ('Wroclaw Akademia Biznesu', 'Poland', None, 'https://wab.edu.pl', 'student.wab.edu.pl,wab.edu.pl')]


In [10]:
DB_FILE = "universities.db"

with sqlite3.connect(DB_FILE) as conn:
    c = conn.cursor()
    
    c.execute('''
        CREATE TABLE universities (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            country TEXT,
            state_province TEXT,
            web_pages TEXT,
            domains TEXT
        )
    ''')
  
    c.executemany("""
        INSERT INTO universities (name, country, state_province, web_pages, domains)
        VALUES (?, ?, ?, ?, ?)
    """, universities_data)
    
    conn.commit()
    
conn.close()

print("ETL concluído! Banco 'universidades.db' criado e populado com sucesso")

OperationalError: table universities already exists

In [11]:


DB_PATH = 'universities.db'

# def query_total_by_country(limit=20):
#     conn = sqlite3.connect(DB_PATH)
#     df = pd.read_sql_query(">
#         SELECT country, COUNT(*) AS total
#         FROM universities
#         GROUP BY country
#         ORDER BY total DESC
#         LIMIT :limit
#     ", conn, params={'limit': limit})
#     conn.close()
#     return df

def query_universities_by_country(country):
    conn = sqlite3.connect(DB_PATH)
    df = pd.read_sql_query("""
        SELECT id, name, state_province, domains, web_pages
        FROM universities
        WHERE lower(country) = lower(:country)
        ORDER BY name
    """, conn, params={'country': country})
    conn.close()
    return df

def search_universities_by_name(term, limit=100):
    conn = sqlite3.connect(DB_PATH)
    term_like = f"%{term}%"
    df = pd.read_sql_query("""
        SELECT id, name, country, domains, web_pages
        FROM universities
        WHERE lower(name) LIKE lower(:term)
        ORDER BY country, name
        LIMIT :limit
    """, conn, params={'term': term_like, 'limit': limit})
    conn.close()
    return df

In [12]:
import sqlite3

DB_FILE = 'universities.db'

with sqlite3.connect(DB_FILE) as conn:
    df_total_por_pais = pd.read_sql_query("""
        SELECT country, COUNT(*) AS total
        FROM universities
        GROUP BY country
        ORDER BY total DESC
    """, conn)
    display(df_total_por_pais)


,country,total
0,United States,2348
1,Japan,572
2,India,473
3,China,397
4,Germany,318
...,...,...
197,Bhutan,1
198,Bermuda,1
199,Barbados,1
200,Bahamas,1
